In [25]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [27]:
epsilon = 0.3
pretrained_model = "model_weights.pth"
use_cuda=True

In [28]:
# MNIST Test dataset and dataloader declaration
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            ])),
        batch_size=1, shuffle=True)

In [29]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

# Initialize the network
model = Net()

# Load the pretrained model
model.load_state_dict(torch.load(pretrained_model))

# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

CUDA Available:  False


Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [42]:
# FGSM attack code
def PGD_Attack(image, label, epsilon, data_grad):
    loss = nn.CrossEntropyLoss()
    image.requires_grad = True

    output = model(image)
    model.zero_grad()
    cost = loss(output, label).to(device)
    cost.backward()

    perturbed_image = image + (2/255) * image.grad.sign()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

In [43]:
def test(model, device, test_loader, epsilon):
    amt_correct = 0
    adv_examples = []

    for data, expected in test_loader:
        data, expected = data.to(device), expected.to(device)
        data.requires_grad = True
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1]
        if init_pred.item() != expected.item():
            continue

        loss = F.nll_loss(output, expected)
        model.zero_grad()
        loss.backward()
        data_grad = data.grad.data
        perturbed_data = PGD_Attack(data, expected, epsilon, data_grad)
        output = model(perturbed_data)

        final_pred = output.max(1, keepdim=True)[1]

        if final_pred.item() == expected.item():
            amt_correct += 1
    
    final_acc = correct/float(len(test_loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(test_loader), final_acc))
    # accuracy = amt_correct/float(len(test_loader))
    # print("At Epsilon: {}".format(epsilon))
    # print("\nThe neural network got {}".format(amt_correct))
    # print("/ {}".format(float(len(test_loader))))
    # print("Or {}".format(accuracy))

In [44]:
test(model, device, test_loader, epsilon)

At Epsilon: 0.3

The neural network got 9525
/ 10000.0
Or 0.9525
